# 🔧 Utilities

Some utilities for demos, used in other notebooks.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import json
import random
from pathlib import Path

import ipywidgets as W
import traitlets as T

import ipyforcegraph.forces as F
from ipyforcegraph.forcegraph import DataFrameSource, ForceGraph

In [ ]:
def make_a_demo(source=None, dataset="datasets/miserables.json", GraphClass=ForceGraph):
    if source is None:
        data = json.loads(Path(dataset).read_text())
        source = DataFrameSource(**data)
    fg = GraphClass(source=source, layout=dict(min_height="500px"))
    style = W.HTML(
        """<style>
        .jp-fg-demo{
            --jp-widgets-container-padding: 0.25em;
            --jp-widgets-inline-width: auto;
        } 
        .widget-box:empty{display:none;}
    </style>"""
    )
    graph_ui = W.VBox()
    node_ui = W.VBox()
    link_ui = W.VBox()
    ui = W.VBox(
        [graph_ui, node_ui, link_ui], layout=dict(width="500px", overflow_y="scroll")
    )
    box = W.HBox([style, fg, ui], layout=dict(height="100%", max_height="100vh"))
    box.add_class("jp-fg-demo")
    box.add_traits(
        behaviors=T.Dict(),
        node_ui=T.Dict(),
        link_ui=T.Dict(),
        graph_ui=T.Dict(),
    )

    def on_box_behaviors(change=None):
        new_behaviors = [b for b in box.behaviors.values() if b is not None]
        fg.behaviors = tuple(new_behaviors)

    box.observe(on_box_behaviors, ["behaviors"])

    T.dlink((box, "graph_ui"), (graph_ui, "children"), lambda d: tuple(d.values()))
    T.dlink((box, "node_ui"), (node_ui, "children"), lambda d: tuple(d.values()))
    T.dlink((box, "link_ui"), (link_ui, "children"), lambda d: tuple(d.values()))
    return fg, box

In [ ]:
def make_an_rgba_picker(**colors):
    kw = {"min": 0, "layout": {"width": "100%"}}
    r, g, b = [
        W.IntSlider(colors.get(x, 0), description=x, max=255, **kw) for x in "rgb"
    ]
    a = W.FloatSlider(colors.get("a", 0), description="a", max=1, **kw)
    sliders = [r, g, b, a]
    box = W.VBox(sliders, layout=dict(width="100%"))
    box.add_traits(color=T.Unicode())

    def update(*args):
        return f"rgba({r.value}, {g.value}, {b.value}, {a.value})"

    [T.dlink((s, "value"), (box, "color"), update) for s in sliders]
    return box

In [ ]:
def make_a_collapsible_picker(title, children):
    ui_children = []
    ui = W.HBox(layout=dict(flex_wrap="wrap"))
    box = W.Accordion([ui], titles=[title])
    select = None
    if len(children) > 1:
        select = W.Dropdown(options=list(children), layout=dict(flex="0"))
        ui.children = [select]
        T.dlink((select, "value"), (ui, "children"), lambda x: [select, *children[x]])
        T.dlink((select, "value"), (box, "titles"), lambda x: (f"{title} ({x})",))
        box.titles = (f"{title} ({select.value})",)
    elif len(children) == 1:
        ui.children = tuple(list(children.values())[0])
    else:
        raise ValueError(f"unexpected number of children {children}")
    box._select = select
    return box

In [ ]:
def make_random_color_series(fg, trait, column_name):
    df = getattr(fg.source, trait)
    digits = "01234567abcdef"
    df[column_name] = [
        "#" + "".join([random.choice(digits) for j in range(6)]) for i in range(len(df))
    ]
    fg.source.send_state(trait)

In [ ]:
def make_link_dropdown_responsive(behavior, label, ui, box):
    box.behaviors = {**box.behaviors, label: None}

    def on_select(change: T.Bunch = None):
        new_behaviors = dict(box.behaviors.items())
        if ui._select.value == "off":
            new_behaviors[label] = None
        else:
            new_behaviors[label] = behavior
        box.behaviors = new_behaviors

    ui._select.observe(on_select, T.All)
    on_select()

In [ ]:
def make_link_behavior_with_ui(WidgetClass, label, column_name, is_color=False):
    def add_behavior(fg, box, column_name=column_name):
        behavior = WidgetClass(column_name=column_name)
        if is_color:
            if column_name not in fg.source.links:
                make_random_color_series(fg, "links", column_name)
        ui_column_name = W.Dropdown(
            options=list(fg.source.links.columns), value=column_name
        )
        ui_template = W.Textarea()
        ui_template_enabled = W.Checkbox(description="enabled?")
        T.link((ui_column_name, "value"), (behavior, "column_name"))
        T.dlink(
            (ui_template, "value"),
            (behavior, "template"),
            lambda x: x if ui_template_enabled.value else "",
        )

        ui = make_a_collapsible_picker(
            label,
            {
                "off": [],
                "column": [ui_column_name],
                "template": [W.VBox([ui_template_enabled, ui_template])],
            },
        )

        box.link_ui = {**box.link_ui, label: ui}

        make_link_dropdown_responsive(behavior, label, ui, box)

        return fg, box

    return add_behavior

In [ ]:
def ensure_graph_forces(fg, box):
    if "graph_forces" not in box.behaviors:
        box.behaviors = {**box.behaviors, "graph_forces": F.GraphForcesBehavior()}

    return box.behaviors["graph_forces"]

In [ ]:
def make_a_force_picker(title, children):
    ui_children = []
    ui = W.HBox(layout=dict(flex_wrap="wrap"))
    box = W.Accordion([ui], titles=[title])
    select = None
    if len(children) > 1:
        options = [(key, value) for key, value in children.items()]
        select = W.Dropdown(options=options, layout=dict(flex="0"))
        ui.children = [select]
        T.dlink((select, "value"), (ui, "children"), lambda x: [select, *select.value])
        T.dlink(
            (select, "value"),
            (box, "titles"),
            lambda x: (f"{title} ({[n for n, v in select.options if v is x][0]})",),
        )
        box.titles = (f"{title} ({select.value})",)
    elif len(children) == 1:
        ui.children = tuple(list(children.values())[0])
    else:
        raise ValueError(f"unexpected number of children {children}")
    box._select = select
    return box

In [ ]:
def ensure_graph_force_ui(fg, box, children: dict):
    if "forces" not in box.graph_ui:
        ui = make_a_force_picker("Forces", {"link": [], "charge": [], "center": []})
        box.graph_ui = {**box.graph_ui, "forces": ui}

    ui = box.graph_ui["forces"]
    existing = {key: child for key, child in ui._select.options}
    for key, child in children.items():
        existing[key] = child

    ui._select.options = [(key, value) for key, value in existing.items()]
    return ui, existing

In [ ]:
import asyncio
from typing import Any


def wait_for_change(widget: W.Widget, value: Any) -> asyncio.Future:
    """Initial pattern from
    https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Asynchronous.html
    """

    future: asyncio.Future = asyncio.Future()

    def getvalue(change: T.Bunch) -> None:
        """make the new value available"""
        future.set_result(change.new)

    def unobserve(f: Any) -> None:
        """unobserves the `getvalue` callback"""
        widget.unobserve(getvalue, value)

    future.add_done_callback(unobserve)

    widget.observe(getvalue, value)

    return future